In [1]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv

In [2]:
# Load Environment Variables
load_dotenv()

True

In [3]:
# Create an engine to connect to the database

from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.asyncio import (
    AsyncSession,
    create_async_engine,
)
db_type="sqlite"

aengine = create_async_engine(
    url=f"{db_type}+aiosqlite:///demo_databases/tn_covid_cases_11_may.sqlite"
)

async_session = sessionmaker(
    bind=aengine,
    class_=AsyncSession,
    expire_on_commit=False
)

In [4]:
# Parameters
metric_db_id = "test"
llm = "gpt-4o"
validation_llm = "gpt-4o"
guardrails_llm = "gpt-4o"

sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities."

db_description = """
[
 {\"name\":\"bed_vacancies_clinics_11_may\",\"description\":\"Each row identifies a district and the beds earmarked, occupied and available for COVID cases in the district clinics.\"},
 {\"name\":\"bed_vacancies_health_centers_and_district_hospitals_11_may\",\"description\":\"Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the district health centers and hospitals.\"},
 {\"name\":\"covid_cases_11_may\",\"description\":\"Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\"}
]
"""

num_common_values = 10

indicator_vars="district_name" # This should be a comma delimited string in multiple vars


### Generate database descriptions and suggested questions for end-users

In [5]:
from askametric.query_processor.db_descriptor.description_generator import get_db_descriptor

db_descr = get_db_descriptor(llm=llm)

async with async_session() as asession:
    description = await db_descr.generate_db_description(
        asession=asession,
        metric_db_id=metric_db_id,
        sys_message=sys_message,
        table_description=db_description,
        column_description=""
)
    suggested_questions = await db_descr.generate_suggested_questions(
        asession=asession,
        metric_db_id=metric_db_id,
        sys_message=sys_message,
        table_description=db_description,
        column_description="",
        # chat_history=chat_history
    )


print(f"DB description:\n {description}\n\n")
print(f"Suggested Questions:\n {suggested_questions}")



DB description:
 The database contains information on COVID-19 management in Tamil Nadu, India, focusing on:

- **Bed Availability:**
  - Clinics: Details on earmarked, occupied, and vacant beds for COVID cases.
  - Health Centers and Hospitals: Information on beds with/without oxygen supply and ICU support, including total vacant beds.

- **COVID Cases:**
  - District-wise data on the number of people under treatment, new cases, discharges, and deaths as of May 11.

The database does not include:

- Detailed patient demographics or individual health records.
- Information on COVID-19 testing rates or vaccination data.
- Financial or logistical data related to healthcare management.
- Data beyond May 11 or projections for future trends.


Suggested Questions:
 ['What is the total number of vacant beds available in clinics across all districts?', 'Which district has the highest number of new COVID-19 cases reported on 11th May?', "How many beds with oxygen supply are currently vacant in

### Single-turn question

In [8]:
# Your question
query = {
    "query_text": "How many beds are available in chennai??",
    "query_metadata": {}
}


In [9]:
from askametric.query_processor.query_processor import LLMQueryProcessor

async with async_session() as session:
    qp = LLMQueryProcessor(
        query,
        session,
        metric_db_id,
        db_type,
        llm,
        guardrails_llm,
        sys_message,
        db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars
    )
    await qp.process_query()
    print(qp.final_answer)

The total number of available beds in Chennai is 4,471. This number is derived from two sources: 3,472 vacant beds from clinics and 999 vacant beds from health centers and district hospitals.


### Multi-turn chat

In [10]:
# Simulating a conversation

queries = [{
    "query_text": "How many beds are available in chennai??",
    "query_metadata": {}
},
{
    "query_text": "How about Ranipet??",
    "query_metadata": {}
},
{
    "query_text": "Inge irrukira COVID patients patthi sollu",
    "query_metadata": {}
},
{
    "query_text": "How many beds with oxygen supply in Ariyalur??",
    "query_metadata": {}
},
{
    "query_text": "And how many COVID patients here?",
    "query_metadata": {}
},
{
    "query_text": "No I want the information for Ariyalur",
    "query_metadata": {}
},
{
    "query_text": "Ranipet aur Ariyalur ke lie phir batao, kya kya jaankari diya hai aapne?",
    "query_metadata": {}
},
{
    "query_text": "What's the distribution of COVID patients in Madurai?",
    "query_metadata": {}
}
]

In [11]:
from askametric.query_processor.query_processor import MultiTurnQueryProcessor

chat_history = []
async with async_session() as session:
    for query in queries:
        print(f"Q: {query['query_text']}")
        mqp = MultiTurnQueryProcessor(
            query=query,
            asession=session,
            metric_db_id=metric_db_id,
            db_type=db_type,
            llm=llm,
            guardrails_llm=guardrails_llm,
            sys_message=sys_message,
            db_description=db_description,
            column_description="",
            indicator_vars=indicator_vars,
            num_common_values=num_common_values,
            chat_history=chat_history
        )
        await mqp.process_query()
        chat_history.append({"user": mqp.reframed_query,
                             "system": mqp.translated_final_answer})
        
        
        print(f"Query type: {mqp.query_type}")
        print(f"Reframed query: {mqp.reframed_query}")
        print(f"A: {mqp.final_answer}")
        print("\n")

Q: How many beds are available in chennai??
Query type: 1
Reframed query: 
A: The total number of available beds in Chennai is 4,471. This number is derived from two sources: 3,472 vacant beds from clinics and 999 vacant beds from health centers and district hospitals.


Q: How about Ranipet??
Query type: 2
Reframed query: What is the total number of available beds in Ranipet?
A: The total number of available beds in Ranipet is 243. This total is derived from two sources: 153 vacant beds from clinics and 90 vacant beds from health centers and district hospitals in Ranipet.


Q: Inge irrukira COVID patients patthi sollu
Query type: 1
Reframed query: 
A: Indha data-la COVID maruthuvamanai padukkai idhalum, maruthuvamanai nodigalil COVID maruthuvam petra manithargalum patri vivarangal irukkirathu. Neengal edhaavadhu kurippaga ketkalaam.


Q: How many beds with oxygen supply in Ariyalur??
Query type: 2
Reframed query: What is the total number of beds with oxygen supply available in Ariyalu